# **Token Classification: Sparse Transfer Learning with the CLI**

In this example, you will fine-tune a 90% pruned BERT model onto some token classification datasets using SparseML's CLI.

### **Sparse Transfer Learning Overview**

Sparse Transfer Learning is very similiar to typical fine-tuning you are used to when training models. However, with Sparse Transfer Learning, we start the training process from a pre-sparsified checkpoint and maintain the sparsity structure while the fine tuning occurs. At the end, you will have a sparse model trained on your dataset, ready to be deployed with DeepSparse for GPU-class performance on CPUs!

### **Pre-Sparsified BERT**
SparseZoo, Neural Magic's open source repository of pre-sparsified models, contains a 90% pruned version of BERT, which has been sparsified on the upstream Wikipedia and BookCorpus datasets with the
masked language modeling objective. [Check out the model card](https://sparsezoo.neuralmagic.com/models/nlp%2Fmasked_language_modeling%2Fobert-base%2Fpytorch%2Fhuggingface%2Fwikipedia_bookcorpus%2Fpruned90-none). We will use this model as the starting point for the transfer learning process.


***Let's dive in!***

## **Installation**

Install SparseML via `pip`.



In [ ]:
%pip uninstall torch torchvision -y
%pip install sparseml[torch]

If you are running on Google Colab, restart the runtime after this step.

In [ ]:
!sparseml.transformers.text_classification --help

## **Sparse Transfer Learning with Conll2003**

SparseML's CLI enables you to kick-off training workflows with various utilities like dataset loading, checkpoint saving, 
metric reporting, and logging handled for you. All we have to do is pass a `model_name_or_path` (the starting checkpoint), a `task` (the GLUE task to train on), and a `recipe` (a YAML file specifying the sparsity related parameters) and we are up and running. The `recipes` is critical for instructing the training script how to modify the training process with sparsity related algorithms. For Sparse Transfer Learning, we will use a `recipe` that instructs SparseML to maintain sparsity during the training process and to apply quantization over the final few epochs. 

### **Run Transfer Learning**

For Conll2003, there is a pre-made transfer learning recipe available in [SparseZoo](https://sparsezoo.neuralmagic.com/models/nlp%2Ftoken_classification%2Fobert-base%2Fpytorch%2Fhuggingface%2Fconll2003%2Fpruned90_quant-none). As such, we kick off transfer learning with the following:

In [ ]:
!sparseml.transformers.train.token_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/pruned90_quant-none \
  --distill_teacher zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/base-none \
  --dataset_name conll2003 \
  --output_dir sparse_bert-token_classification_conll2003 \
  --per_device_train_batch_size 32 --per_device_eval_batch_size 32 --preprocessing_num_workers 6 \
  --do_train --do_eval --evaluation_strategy epoch --fp16 --seed 29204  \
  --save_strategy epoch --save_total_limit 1 \
  --max_train_samples 2000

Let's discuss the key arguments:
- `--dataset_name conll2003` instructs SparseML to download and fine-tune onto the Conll2003 dataset. The script automatically downloads the dataset from the Hugging Face hub.

- `--model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none` specifies the starting checkpoint for the fine tuning. Here, we passed a SparseZoo stub identifying the 90% pruned version of BERT trained with masked language modeling on the Wikipedia and BookCorpus datasets. SparseML downloads the checkpoint from the Zoo when the script starts.

- `--recipe zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/pruned90_quant-none` specifies the recipe to be applied by SparseML. Here, we passed a SparseZoo stub identifying the transfer learning recipe for the Conll2003 dataset. SparseML downloads the recipe from the Zoo when the script starts. See below for the details of what this recipe looks like.

- `--distill_teacher zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/base-none` is an optional argument that specifies a model to use for as a teacher to apply distillation during the training process. We passed a SparseZoo stub identifying a dense BERT model trained on Conll2003. SparseML downloads the teacher from the Zoo when the script starts.

The script downloads the starting checkpoint, the teacher model, and transfer learning recipe from SparseZoo as well as the Conll2003 dataset and trains the model for 13 epochs, converging to ~98.5% accuracy on the validation set. The final model is quantized with 90% of weights pruned!

#### **Transfer Learning Recipe**

Here's what the transfer learning recipe for the Conll2003 dataset looks like.

The "Modifiers" are the important items that encode how SparseML should modify the training process for Sparse Transfer Learning:
- `ConstantPruningModifier` tells SparseML to pin weights at 0 over all epochs, maintaining the sparsity structure of the network
- `QuantizationModifier` tells SparseML to quanitze the weights with quantization aware training over the last 5 epochs
- `DistillationModifier` tells SparseML how to apply distillation to the model, including the layer and some hyperparameters

SparseML parses the modifiers and updates the training process to implement the algorithms and hyperparameters specified in the recipes.

In [ ]:
from sparsezoo import Model
transfer_stub = "zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/pruned90_quant-none"
download_dir = "./transfer_recipe"
zoo_model = Model(transfer_stub, download_path=download_dir)
recipe_path = zoo_model.recipes.default.path
print(recipe_path)

In [ ]:
%cat ./transfer_recipe/recipe/recipe_original.md

### **Export to ONNX**

Once you have trained your model, export to ONNX in order to deploy with DeepSparse. The artifacts of the training process 
are saved to your local filesystem. 

Run the following to convert your PyTorch checkpoint to ONNX:

In [ ]:
!sparseml.transformers.export_onnx \
  --model_path ./sparse_bert-token_classification_conll2003 \
  --task text_classification

The script above creates a `deployment` folder in your local directory, which has all of the files needed for deployment with DeepSparse including the `model.onnx`, `config.json`, and `tokenizer.json` files.

## **Sparse Transfer Learning with a Custom Dataset**

Beyond the Conll2003 dataset, we can also use a dataset from the Hugging Face Hub or pass via local files. Let's try an example of each for the sentiment analysis using [WNUT 17](wnut_17), which is also a NER task.

For simplicity, we will perform the fine-tuning without distillation. Although the transfer learning recipe contains distillation
modifiers, by setting `--distill_teacher disable` we instruct SparseML to skip distillation.

### **Using a Hugging Face Dataset**

Let's walk through how to pass a Hugging Face dataset identifier to the CLI.

In [ ]:
from datasets import load_dataset
from pprint import pprint

wnut_17 = load_dataset("wnut_17")
print(wnut_17)
print(wnut_17["train"][0])

#### **Kick off Training**

To use this dataset with the CLI, we can replace the `--dataset_name conll2003` argument with `--dataset_name wnut_17 --input_column_names tokens --label_column_name label`. SparseML will then download the dataset from the Hugging Face hub and run training as before.

In [ ]:
!sparseml.transformers.token_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/pruned90_quant-none \
  --recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}' \
  --distill_teacher disable \
  --dataset_name wnut_17 --text_column_name tokens --label_column_name ner_tags \
  --output_dir sparse_bert-token_classification_wnut_17 \
  --per_device_train_batch_size 32 --per_device_eval_batch_size 32 --preprocessing_num_workers 6 \
  --do_train --do_eval --evaluation_strategy epoch --fp16 --seed 29204  \
  --save_strategy epoch --save_total_limit 1

You will notice that we used the same recipe as we did in the Conll2003 case (identified by the SparseZoo stub `zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/pruned90_quant-none`). Since the WNUT and Conll2003 tasks are similiar, we chose to start with the same hyperparameters as we used in Conll2003 training.


To update a recipe, you can download the YAML file from SparseZoo, make updates to the YAML directly, and pass the local path to SparseML. In this case, we used `--recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}'` to modify a recipe on the fly, updating to only run for 11 epochs.

### **Using Local JSON Files**

Let's walk through how to pass a JSON dataset to the CLI.

#### **Save Dataset as a JSON File**

For this example, we use Hugging Face `datasets` to create a JSON file for WNUT 17 that can be passed to SparseML's CLI.

In [ ]:
from datasets import load_dataset
from pprint import pprint

dataset = load_dataset("wnut_17")
print(dataset)
print(dataset["train"][0])

In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names
print(label_list)

For the NER task, the input in our JSON files should be labeled

In [ ]:
named_labels = []
for i in range(len(dataset["train"])):
  named_labels_i = [label_list[label_idx] for label_idx in dataset["train"][i]["ner_tags"]]
  named_labels.append(named_labels_i)

eval_named_labels = []
for i in range(len(dataset["validation"])):
  named_labels_i = [label_list[label_idx] for label_idx in dataset["validation"][i]["ner_tags"]]
  eval_named_labels.append(named_labels_i)

dataset["train"] = dataset["train"].add_column("named_ner_tags", named_labels)
dataset["validation"] = dataset["validation"].add_column("named_ner_tags", eval_named_labels)
dataset["train"] = dataset["train"].remove_columns("ner_tags")
dataset["validation"] = dataset["validation"].remove_columns("ner_tags")

In [ ]:
dataset["train"].to_json("./wnut_17-train.json")
dataset["validation"].to_json("./wnut_17-validation.json")

We can see that the data is a JSON file with `tokens` and `named_ner_tags`. 

As described above, the token classification trianing pipeline expects the input sequences to be a lists of words with a tag for each word.

When passing the dataset via local files, the tags must be strings representing each class.

In [ ]:
!head ./wnut_17-train.json --lines=5

In [ ]:
!head ./wnut_17-validation.json --lines=5

#### **Kick off Training**

To use the local files with the CLI, pass `--train_file ./wnut_17-train.json --validation_file ./wnut_17-validation.json  --text_column_name tokens --label_column_name named_ner_tags`.

Run the following:

In [ ]:
!sparseml.transformers.token_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/token_classification/obert-base/pytorch/huggingface/conll2003/pruned90_quant-none \
  --distill_teacher disable \
  --train_file wnut_17-train.json --validation_file wnut_17-validation.json \
  --text_column_name tokens --label_column_name named_ner_tags \
  --output_dir sparse_bert-token_classification_wnut_17_from_json \
  --per_device_train_batch_size 32 --per_device_eval_batch_size 32 --preprocessing_num_workers 6 \
  --do_train --do_eval --evaluation_strategy epoch --fp16 --seed 29204  \
  --save_strategy epoch --save_total_limit 1